<a href="https://colab.research.google.com/github/advik-7/Agents/blob/main/Fetch_Attendance_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 27.8 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
import json
from langchain.tools import tool

def fetch_attendance(username: str, password: str) -> dict:
    """
    Logs into the Moodle using provided credentials and fetches attendance details using Selenium.
    """

    try:


        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--headless')  # Run Chrome in headless mode
        chrome_options.add_argument('--no-sandbox')  # Bypass OS security model
        chrome_options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems

        # Initialize WebDriver with Chrome options
        driver = webdriver.Chrome(options=chrome_options)

        driver.get("https://lms.sitpune.edu.in/login/index.php")




        username = driver.find_element(By.NAME, "username")

        password = driver.find_element(By.NAME, "password")

        username.send_keys("23070126007")
        password.send_keys("Nightwing@126789")


        password.send_keys(Keys.RETURN)

        time.sleep(3)

        # Navigate to the DPEL course page
        driver.find_element(By.XPATH, "//a[@href='https://lms.sitpune.edu.in/course/view.php?id=1246' and contains(@class, 'aalink') and contains(@class, 'coursename')]").click()
        time.sleep(3)

        # Open the attendance activity
        driver.find_element(By.CSS_SELECTOR, "div.activitytitle.media.modtype_attendance").click()
        time.sleep(3)

        # Navigate to 'All sessions'
        driver.find_element(By.CSS_SELECTOR, "a.nav-link[title='All sessions']").click()
        time.sleep(2)

        # Open the attendance details page
        try:
            driver.find_element(By.CSS_SELECTOR, 'a[href="https://lms.sitpune.edu.in/mod/attendance/view.php?mode=2&id=6836&view=5"]').click()
        except NoSuchElementException:
            print("Failed to find the attendance details link.")
            return {}

        time.sleep(2)

        # Fetch attendance data
        totals = {}
        totals_locator = (By.XPATH, "//table[@class='generaltable table-bordered table-condensed']//tr")
        points_locator = (By.XPATH, "//tr[@class='lastrow']//td")

        rows = driver.find_elements(*totals_locator)
        for row in rows[1:]:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) == 2:
                key = cells[0].text.strip()
                value = cells[1].text.strip()
                totals[key] = value

        points_text = driver.find_element(*points_locator).text.strip()
        totals['Points'] = points_text

        return totals

    finally:
        # Close the browser
        driver.quit()


# Usage example
attendance_details = fetch_attendance("username", "password")
for key, value in attendance_details.items():
    print(f"{key}: {value}")


Present:: 173
Late:: 0
Excused:: 0
Absent:: 97
Points: Points: 346/540 (64.1%) to date
